In [7]:
import os
from datetime import date
from dotenv import load_dotenv
import pandas as pd

from nowcast_data.pit.api import PITDataManager
from nowcast_data.pit.core.catalog import SeriesCatalog

# Load environment variables from .env file
load_dotenv()


"""
This example demonstrates how to get point-in-time data for US GDP (US_GDP_SAAR).
The output is a pandas Series with a multi-index, where the first level is the
vintage date (the date the data was available) and the second level is the
observation date (the period the data refers to).
"""

if "FRED_API_KEY" not in os.environ:
    print("FRED_API_KEY environment variable not set. Please set it in your .env file.")
    raise ValueError()

# Load the series catalog
catalog_path = os.path.join("..", "series_catalog.yaml")
catalog = SeriesCatalog(catalog_path)
data_manager = PITDataManager(catalog=catalog)

series_key = "US_GDP_SAAR"

# 1. Get all vintage dates for the series
print(f"Fetching vintage dates for {series_key}...")
vintage_dates = data_manager.get_series_vintages(series_key)
print(f"Found {len(vintage_dates)} vintage dates.")

# For this example, we will use the last 5 vintages to keep the output small
sample_vintages = vintage_dates[-5:]
print(f"Fetching data for the last {len(sample_vintages)} vintages...")

all_series = []
for vintage in sample_vintages:
    print(f"  Fetching data for vintage: {vintage}")
    df = data_manager.get_series_asof(series_key, vintage)
    # The 'asof_date' column represents the vintage date
    # The 'obs_date' column represents the reference period
    df = df.set_index(["asof_date", "obs_date"])["value"]
    df.index = df.index.set_levels(pd.to_datetime(df.index.levels[1]).to_period('Q'), level=1)
    all_series.append(df)

# Concatenate all series into a single multi-indexed series
result_series = pd.concat(all_series)

print("\n--- US GDP Point-in-Time Data (Sample) ---")
print(result_series)

# You can also reshape the data to have vintages as columns
print("\n--- US GDP Point-in-Time Data (Reshaped) ---")
print(result_series.unstack(level=0))


Fetching vintage dates for US_GDP_SAAR...
Found 405 vintage dates.
Fetching data for the last 5 vintages...
  Fetching data for vintage: 2025-06-26
  Fetching data for vintage: 2025-07-30
  Fetching data for vintage: 2025-08-28
  Fetching data for vintage: 2025-09-25
  Fetching data for vintage: 2025-12-23

--- US GDP Point-in-Time Data (Sample) ---
asof_date   obs_date
2025-06-26  1946Q1            NaN
            1946Q2            NaN
            1946Q3            NaN
            1946Q4            NaN
            1947Q1        243.164
                          ...    
2025-12-23  2024Q3      29511.664
            2024Q4      29825.182
            2025Q1      30042.113
            2025Q2      30485.729
            2025Q3      31095.089
Name: value, Length: 1590, dtype: float64

--- US GDP Point-in-Time Data (Reshaped) ---
asof_date  2025-06-26  2025-07-30  2025-08-28  2025-09-25  2025-12-23
obs_date                                                             
1946Q1            NaN    

In [8]:
result_series.unstack(level=0)

asof_date,2025-06-26,2025-07-30,2025-08-28,2025-09-25,2025-12-23
obs_date,,,,,
1946Q1,NaN,NaN,NaN,NaN,NaN
1946Q2,NaN,NaN,NaN,NaN,NaN
1946Q3,NaN,NaN,NaN,NaN,NaN
1946Q4,NaN,NaN,NaN,NaN,NaN
1947Q1,243.164,243.164,243.164,243.164,243.164
...,...,...,...,...,...
2024Q3,29374.914,29374.914,29374.914,29511.664,29511.664
2024Q4,29723.864,29723.864,29723.864,29825.182,29825.182
2025Q1,29962.047,29962.047,29962.047,30042.113,30042.113


In [9]:
vintage_dates

[datetime.date(1991, 12, 4),
 datetime.date(1991, 12, 20),
 datetime.date(1992, 1, 29),
 datetime.date(1992, 2, 28),
 datetime.date(1992, 3, 26),
 datetime.date(1992, 4, 28),
 datetime.date(1992, 5, 29),
 datetime.date(1992, 6, 25),
 datetime.date(1992, 7, 30),
 datetime.date(1992, 8, 27),
 datetime.date(1992, 9, 24),
 datetime.date(1992, 10, 27),
 datetime.date(1992, 11, 25),
 datetime.date(1992, 12, 22),
 datetime.date(1993, 1, 28),
 datetime.date(1993, 2, 26),
 datetime.date(1993, 3, 26),
 datetime.date(1993, 4, 29),
 datetime.date(1993, 5, 28),
 datetime.date(1993, 6, 23),
 datetime.date(1993, 7, 29),
 datetime.date(1993, 8, 31),
 datetime.date(1993, 9, 29),
 datetime.date(1993, 10, 28),
 datetime.date(1993, 12, 1),
 datetime.date(1993, 12, 22),
 datetime.date(1994, 1, 28),
 datetime.date(1994, 3, 1),
 datetime.date(1994, 3, 31),
 datetime.date(1994, 4, 28),
 datetime.date(1994, 5, 27),
 datetime.date(1994, 6, 29),
 datetime.date(1994, 7, 29),
 datetime.date(1994, 8, 26),
 datetime